# Fragrance Data Cleaning Pipeline


## Objectives
- Transform raw fragrance data into a clean, analysis-ready dataset
- Handle missing values and incorrect data types
- Normalize categorical variables
- Remove duplicate records
- Parse complex text fields

## Load Raw Data


In [61]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/fra_perfumes.csv")


## Handle Missing Values


In [62]:
# Track missing ratings
df['rating_missing'] = df['Rating Value'].isna()

# No ratings keep them but fill with 0
df['Rating Value'] = df['Rating Value'].fillna(0)
df['Rating Count'] = df['Rating Count'].fillna(0)

# Remove records lacking critical identity fields
df = df.dropna(subset=['Name', 'Gender', 'Description'])

# Verification
df[['Rating Value','Rating Count']].isna().sum()



Rating Value    0
Rating Count    0
dtype: int64

### Missing Data Strategy
- Kept records with missing ratings to avoid unnecessary data loss
- Filled missing rating values and counts with 0 to represent unrated fragrances
- Removed records without needed identifying information

## Fix Data Types



In [63]:
# Clean Rating Count
df['Rating Count'] = (
    df['Rating Count']
    .astype(str)
    .str.replace(',', '', regex=False)
    .str.replace('k', '000', regex=False)
)

df['Rating Count'] = pd.to_numeric(df['Rating Count'], errors='coerce').fillna(0).astype(int)

# Verification
df[['Rating Value','Rating Count']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 70100 entries, 0 to 70102
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rating Value  70100 non-null  float64
 1   Rating Count  70100 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 1.6 MB


### Data Type Corrections
- Converted rating counts from strings into numeric format
- Handled non-numeric and malformed values safely using coercion
- Ensured rating metrics are usable for analysis

## Normalize Categories


In [64]:
# --- Normalize Gender ---

# Inspect original values
print(df['Gender'].value_counts(dropna=False))

# Clean casing and whitespace
df['Gender'] = df['Gender'].str.lower().str.strip()

# Map to final categories
gender_map = {
    'for women': 'Women',
    'for men': 'Men',
    'for women and men': 'Unisex'
}

df['Gender'] = df['Gender'].replace(gender_map)

# Verify
print(df['Gender'].value_counts(dropna=False))

# --- Clean Name field (after Gender is fixed) ---

df['Name'] = (
    df['Name']
    .str.replace(' for women and men', '', case=False, regex=False)
    .str.replace(' for women', '', case=False, regex=False)
    .str.replace(' for men', '', case=False, regex=False)
    .str.strip()
)

# Add a space to 'for ...' in Name
df['Name'] = df['Name'].str.replace('for women and men', ' for women and men', regex=False)
df['Name'] = df['Name'].str.replace('for women', ' for women', regex=False)
df['Name'] = df['Name'].str.replace('for men', ' for men', regex=False)

# Remove gender phrases from Name
df['Name'] = (
    df['Name']
    .str.replace(' for women and men', '', case=False, regex=False)
    .str.replace(' for women', '', case=False, regex=False)
    .str.replace(' for men', '', case=False, regex=False)
    .str.strip()
)

# Final check
df[['Name','Gender']].head(10)

Gender
for women and men    29708
for women            28102
for men              12290
Name: count, dtype: int64
Gender
Unisex    29708
Women     28102
Men       12290
Name: count, dtype: int64


,Name,Gender
0,9am Afnan,Women
1,9am Dive Afnan,Unisex
2,9am pour Femme Afnan,Women
3,9pm Afnan,Men
4,9pm pour Femme Afnan,Women
5,Naseej Al Kiswah Afnan,Unisex
6,Naseej Al Oud Afnan,Unisex
7,Naseej Al Ward Afnan,Unisex
8,Naseej Al Zafaran Afnan,Unisex
9,Afzal Abeer Afnan,Women


### Category Normalization
- Standardized gender labels into three categories: Women, Men, and Unisex
- Improved consistency for analysis and grouping

## Remove Duplicates


In [65]:
# duplicates
df.duplicated(subset=['Name', 'url']).sum()

# Remove duplicates using business logic
before = len(df)
df = df.drop_duplicates(subset=['Name', 'url'], keep='first')
after = len(df)
before, after, before - after


(70100, 69945, 155)

### Duplicate Handling
- Identified duplicates using Name and URL as a key
- Kept the first occurrence of each unique fragrance entry
- Removed 155 duplicate records to improve integrity

## Parse Complex Fields


In [66]:
df['Main Accords'].head(10)

import ast

def parse_listlike(x):
    if pd.isna(x):
        return []
    x = str(x).strip()
    if x in ("", "[]"):
        return []
    try:
        val = ast.literal_eval(x)
        return val if isinstance(val, list) else []
    except Exception:
        return []

# Convert into real lists
df['main_accords_list'] = df['Main Accords'].apply(parse_listlike)

# Create Main Accord columns
for i in range(5):
    df[f"Main Accord {i+1}"] = df['main_accords_list'].apply(lambda lst: lst[i] if len(lst) > i else np.nan)

df[['Main Accords', 'Main Accord 1', 'Main Accord 2', 'Main Accord 3']].head(10)

,Main Accords,Main Accord 1,Main Accord 2,Main Accord 3
0,"['citrus', 'musky', 'woody', 'aromatic', 'warm...",citrus,musky,woody
1,"['fruity', 'woody', 'green', 'warm spicy', 'ar...",fruity,woody,green
2,"['fruity', 'musky', 'amber', 'citrus', 'powder...",fruity,musky,amber
3,"['vanilla', 'amber', 'warm spicy', 'cinnamon',...",vanilla,amber,warm spicy
4,"['woody', 'aromatic', 'rose', 'fruity', 'powde...",woody,aromatic,rose
5,"['woody', 'amber', 'leather', 'oud', 'patchoul...",woody,amber,leather
6,"['oud', 'leather', 'woody', 'amber', 'soft spi...",oud,leather,woody
7,"['rose', 'oud', 'fruity', 'musky', 'sweet', 'f...",rose,oud,fruity
8,"['warm spicy', 'metallic', 'leather', 'fresh s...",warm spicy,metallic,leather
9,[],NaN,NaN,NaN


### Main Accords Parsing
- Converted string list values in `Main Accords` into lists
- Split the top accords into structured columns (Main Accord 1–5)
- Handled empty lists and malformed entries safely

In [67]:
# Remove non-informative columns
df_final = df.drop(columns=['Perfumers'])

# Remove helper column from parsing
if 'main_accords_list' in df_final.columns:
    df_final = df_final.drop(columns=['main_accords_list'])

# Final dataset shape and missing values check
print("Final shape:", df_final.shape)
print("\nMissing values (top 10):")
print(df_final.isna().sum().sort_values(ascending=False).head(10))

# Export cleaned dataset
df_final.to_csv("../data/processed/cleaned_fragrances.csv", index=False)

Final shape: (69945, 13)

Missing values (top 10):
Main Accord 5    9053
Main Accord 4    5714
Main Accord 3    3287
Main Accord 2    1875
Main Accord 1    1434
Main Accords        0
Rating Count        0
Rating Value        0
Gender              0
Name                0
dtype: int64


### Final Notes
- The `Perfumers` column was removed because it contained no populated values across the dataset.
- Brand information is enough for analysis purposes.

## Final Validation & Export


In [68]:
# Remove all perfumer-related columns completely
df_final = df.drop(columns=['Perfumers', 'Perfumer1', 'Perfumer2', 'Perfumer3'], errors='ignore')

# Remove helper column if still present
df_final = df_final.drop(columns=['main_accords_list'], errors='ignore')

# Final checks
print("Final shape:", df_final.shape)
print("\nMissing values (top 10):")
print(df_final.isna().sum().sort_values(ascending=False).head(10))

# Export cleaned dataset
df_final.to_csv("../data/processed/cleaned_fragrances.csv", index=False)


Final shape: (69945, 13)

Missing values (top 10):
Main Accord 5    9053
Main Accord 4    5714
Main Accord 3    3287
Main Accord 2    1875
Main Accord 1    1434
Main Accords        0
Rating Count        0
Rating Value        0
Gender              0
Name                0
dtype: int64
